# Simulating follow-on interviews with survey respondents
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for creating AI agents representing survey respondents and simulating follow-on interviews with them.

[EDSL is an open-source libary](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Importing survey data
We start by using the `Conjure` module to import a dataset of survey responses and convert it into EDSL objects. 
For purposes of demonstration we'll use a set of mock responses to a reader survey about a home renovation newsletter:

In [1]:
# "How often do you read our home renovation tips newsletter?","On a scale of 1-10, how useful do you find the content in our newsletter? (1 = Not useful at all, 10 = Extremely useful)","Would you recommend our newsletter to a friend or family member?","What topics would you like to see covered in future issues of our newsletter?"
# "Weekly",9,"Yes","More articles on upcycling old furniture, please! I'm all about that eco-friendly life."
# "Monthly",7,"Yes","How about some tips for renovating tiny apartments? Not all of us live in mansions, you know!"
# "Quarterly",4,"No","Honestly, I find most of the content too advanced. Could you include a 'Renovation 101' section for us newbies?"
# "Rarely",2,"No","Your newsletter is way too text-heavy. Ever heard of infographics? Or videos? Join the 21st century, folks!"
# "Weekly",10,"Yes","I'd kill for a deep dive into historical home restoration techniques. Let's get nerdy with it!"
# "Monthly",6,"No","Meh, it's okay. Maybe feature some celebrity home renos? That'd spice things up a bit."
# "Quarterly",8,"Yes","As a contractor, I'd love to see a section addressing common client misconceptions about renovations."
# "Weekly",9,"Yes","Your content rocks, but how about some love for us renters? Removable upgrades are my jam!"
# "Monthly",5,"No","Too many ads, not enough substance. I feel like I'm reading a catalog half the time."
# "Rarely",3,"No","Snooze fest. Where's the excitement? How about some extreme home makeover challenges?"
# "Weekly",10,"Yes","I'm obsessed with smart home tech. Can we get more articles on integrating tech into older homes?"
# "Monthly",7,"Yes","Love the newsletter, but please proofread! The typos are driving my inner grammar nerd crazy."
# "Quarterly",6,"Yes","How about a series on renovation horror stories and how to avoid them? Could be both helpful and entertaining!"
# "Rarely",1,"No","Unsubscribe button, where art thou? This newsletter is about as useful as a chocolate teapot."
# "Monthly",8,"Yes","Any chance of including a Q&A section with real architects or designers? I've got questions!"
# "Weekly",9,"Yes","More budget breakdowns, please! I want to know exactly where my renovation dollars are going."
# "Quarterly",5,"No","It's fine, I guess. But could you cover more international design trends? It's a big world out there!"
# "Monthly",7,"Yes","Hey, how about some tips for pet-friendly renovations? My cats are destroying everything!"
# "Weekly",10,"Yes","Absolutely adore your content! But I'm dying to see more about incorporating art into home design. Let's get creative!"
# "Rarely",4,"No","Too mainstream for my taste. Where are the articles on avant-garde design or extreme home modifications?"

We've saved the CSV at the Coop and can re-import it here:

In [2]:
from edsl.scenarios.FileStore import CSVFileStore

In [3]:
csv_file = CSVFileStore.pull('fa512b3e-f060-49f8-b7bf-bf20b1132ab2', expected_parrot_url='https://www.expectedparrot.com')

In [4]:
# Code for uploading a CSV to the Coop:

# refresh = False
# if refresh:
#     from edsl.scenarios.FileStore import CSVFileStore
#     fs = CSVFileStore("newsletter_survey_responses.csv")
#     info = fs.push()
#     print(info)

Conjure allows us to recreate the file as EDSL objects ([learn more about using Conjure](https://docs.expectedparrot.com/en/latest/conjure.html)):

In [5]:
from edsl import Conjure

In [6]:
c = Conjure(csv_file.to_tempfile())

We can inspect the questions that were in the survey:

In [7]:
c.question_texts

['How often do you read the newsletter?',
 'What topics would you like to see covered in future issues?',
 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?',
 'Which section of the newsletter do you find most valuable?',
 'What improvements would you suggest for the newsletter?']

## Generating EDSL objects
The `Conjure` module provides methods for automatically generating `Survey`, `Results` and `Agent` objects from the survey data. These methods allow us to use [built-in methods for analyzing the results](https://docs.expectedparrot.com/en/latest/results.html) and [create AI agents](https://docs.expectedparrot.com/en/latest/agents.html) representing the original survey respondents.

### Recreating results
For example, we can recreate the responses as EDSL `Results` and sample, filter, select and print them:

In [8]:
results = c.to_results()

In [9]:
(results
 .sample(3)
 .select("answer.*")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ answer               ┃ answer               ┃ answer               ┃ answer               ┃ answer              ┃
┃ .topics_would_like_… ┃ .section_newsletter… ┃ .improvements_would… ┃ .scale_1_10_would_r… ┃ .often_read_newsle… ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ Case studies on      │ Case Studies         │ More in-depth        │ 9                    │ Daily               │
│ innovative           │                      │ articles on new      │                      │                     │
│ construction         │                      │ construction         │                      │                     │
│ techniques are       │                      │ technologies and     │                      │                     │
│ always interesting.  │                      │ materials would be   │                      │                     │
│ Detailed breakdowns  │                      │ beneficial.          │                      │                     │
│ of successful        │                      │ Including            │                      │                     │
│ projects, including  │                      │ comparisons of       │                      │                     │
│ the challenges faced │                      │ different products   │                      │                     │
│ and solutions        │                      │ and their            │                      │                     │
│ implemented, would   │                      │ applications would   │                      │                     │
│ be great. Interviews │                      │ provide valuable     │                      │                     │
│ with the             │                      │ insights. A section  │                      │                     │
│ construction teams   │                      │ dedicated to         │                      │                     │
│ would add a personal │                      │ frequently asked     │                      │                     │
│ touch, making the    │                      │ questions and expert │                      │                     │
│ stories more         │                      │ advice would be a    │                      │                     │
│ relatable and        │                      │ valuable addition.   │                      │                     │
│ informative.         │                      │                      │                      │                     │
├──────────────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ I would love to see  │ Construction Tips    │ Adding more video    │ 8                    │ Weekly              │
│ more in-depth guides │                      │ content, such as     │                      │                     │
│ on sustainable       │                      │ tutorials and        │                      │                     │
│ building practices.  │                      │ walk-throughs, would │                      │                     │
│ Including case       │                      │ make the newsletter  │                      │                     │
│ studies from         │                      │ more engaging.       │                      │                     │
│ different types of   │                      │ Summaries of key     │                      │                     │
│ projects would be    │                      │ points at the end of │                      │                     │
│ very helpful.        │                      │ each issue would     │                      │                     │
│ Interviews with      │                      │ also be beneficial.  │                      │                     │
│ experts about their  │                      │ It would

### Designing AI agents
We can also generate an `AgentList` for the respondents, each containing a dictionary of `traits` for a respondent's survey answers and a `codebook` for the questions that were asked:

In [10]:
agents = c.to_agent_list()

We can inspect some of them:

In [11]:
agents[0:2]

AgentList([Agent(traits = {'often_read_newsletter': 'Weekly', 'topics_would_like_see_covered': 'I would love to see more in-depth guides on sustainable building practices. Including case studies from different types of projects would be very helpful. Interviews with experts about their experiences, tips, and challenges they faced would also be appreciated. Additionally, insights into cost-saving techniques would be very beneficial.', 'scale_1_10_would_rate_overall': 8, 'section_newsletter_find_valuable': 'Construction Tips', 'improvements_would_suggest': 'Adding more video content, such as tutorials and walk-throughs, would make the newsletter more engaging. Summaries of key points at the end of each issue would also be beneficial. It would also be great to include downloadable templates and checklists for various construction projects.'}, codebook = {'often_read_newsletter': 'How often do you read the newsletter?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues?', 'scale_1_10_would_rate_overall': 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?', 'section_newsletter_find_valuable': 'Which section of the newsletter do you find most valuable?', 'improvements_would_suggest': 'What improvements would you suggest for the newsletter?'}), Agent(traits = {'often_read_newsletter': 'Monthly', 'topics_would_like_see_covered': "I think the newsletter could benefit from a series on advanced home automation techniques. Tutorials on installing smart home devices and interviews with professionals in the field would be very useful. Additionally, I'd love to see articles on integrating renewable energy solutions into home construction.", 'scale_1_10_would_rate_overall': 7, 'section_newsletter_find_valuable': 'Product Reviews', 'improvements_would_suggest': 'The layout of the newsletter could be improved with more visual elements like diagrams and photos. Simplifying the design for easier readability would enhance the overall experience. Including interactive elements like polls and Q&A sections would also add value.'}, codebook = {'often_read_newsletter': 'How often do you read the newsletter?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues?', 'scale_1_10_would_rate_overall': 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?', 'section_newsletter_find_valuable': 'Which section of the newsletter do you find most valuable?', 'improvements_would_suggest': 'What improvements would you suggest for the newsletter?'})])

We can give the agents names for reference in conducting follow-on interviews:

In [12]:
for i, agent in enumerate(agents):
    setattr(agent, 'name', f"Respondent {i+1}")

In [13]:
agents[0:2]

AgentList([Agent(name = """Respondent 1""", traits = {'often_read_newsletter': 'Weekly', 'topics_would_like_see_covered': 'I would love to see more in-depth guides on sustainable building practices. Including case studies from different types of projects would be very helpful. Interviews with experts about their experiences, tips, and challenges they faced would also be appreciated. Additionally, insights into cost-saving techniques would be very beneficial.', 'scale_1_10_would_rate_overall': 8, 'section_newsletter_find_valuable': 'Construction Tips', 'improvements_would_suggest': 'Adding more video content, such as tutorials and walk-throughs, would make the newsletter more engaging. Summaries of key points at the end of each issue would also be beneficial. It would also be great to include downloadable templates and checklists for various construction projects.'}, codebook = {'often_read_newsletter': 'How often do you read the newsletter?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues?', 'scale_1_10_would_rate_overall': 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?', 'section_newsletter_find_valuable': 'Which section of the newsletter do you find most valuable?', 'improvements_would_suggest': 'What improvements would you suggest for the newsletter?'}), Agent(name = """Respondent 2""", traits = {'often_read_newsletter': 'Monthly', 'topics_would_like_see_covered': "I think the newsletter could benefit from a series on advanced home automation techniques. Tutorials on installing smart home devices and interviews with professionals in the field would be very useful. Additionally, I'd love to see articles on integrating renewable energy solutions into home construction.", 'scale_1_10_would_rate_overall': 7, 'section_newsletter_find_valuable': 'Product Reviews', 'improvements_would_suggest': 'The layout of the newsletter could be improved with more visual elements like diagrams and photos. Simplifying the design for easier readability would enhance the overall experience. Including interactive elements like polls and Q&A sections would also add value.'}, codebook = {'often_read_newsletter': 'How often do you read the newsletter?', 'topics_would_like_see_covered': 'What topics would you like to see covered in future issues?', 'scale_1_10_would_rate_overall': 'On a scale of 1 to 10, how would you rate the overall quality of the newsletter?', 'section_newsletter_find_valuable': 'Which section of the newsletter do you find most valuable?', 'improvements_would_suggest': 'What improvements would you suggest for the newsletter?'})])

## Conducting follow-on interviews
We can use the `Conversation` module to automate an interview with each respondent agent. 
To start, we can create an agent interviewer and pair it with each respondent agent:

In [14]:
from edsl import AgentList, Agent
from edsl.conversation.Conversation import Conversation, ConversationList

In [15]:
def interview_pair(i, agents):
    respondent = agents[i]
    interviewer = Agent(
        name = "Interviewer",
        traits = {
            "motivation": f"""
            You are conducting a follow-on interview with someone who has just 
            answered a survey about a home construction newsletter. 
            You want to get more details about their feedback on the newsletter.
            Here are the survey questions and the respondent's answers:
            { respondent.codebook }
            { respondent.traits }
            """
        }
    )
        
    return AgentList([interviewer, respondent])

Next we specify the agents and number of turns for each conversation that we want to run:

In [16]:
cl = ConversationList(
    [Conversation(agent_list = interview_pair(i, agents), max_turns = 6)
    for i in range(len(agents))]
)

Now we can run the conversations and convert them into `Results` objects for inspection and analysis:

In [17]:
cl.run()

results = cl.to_results()

To see all the components of the results that have been generated:

In [18]:
results.columns

['agent.agent_instruction',
 'agent.improvements_would_suggest',
 'agent.motivation',
 'agent.often_read_newsletter',
 'agent.scale_1_10_would_rate_overall',
 'agent.section_newsletter_find_valuable',
 'agent.topics_would_like_see_covered',
 'answer.dialogue',
 'comment.dialogue_comment',
 'generated_tokens.dialogue_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.dialogue_system_prompt',
 'prompt.dialogue_user_prompt',
 'question_options.dialogue_question_options',
 'question_text.dialogue_question_text',
 'question_type.dialogue_question_type',
 'raw_model_response.dialogue_cost',
 'raw_model_response.dialogue_one_usd_buys',
 'raw_model_response.dialogue_raw_model_response',
 'scenario.agent_name',
 'scenario.conversation',
 'scenario.conversation_index',
 'scenario.index']

As we did above, we can filter, sort, select and print the results. 
Please see documentation for [examples of other analytical methods](https://docs.expectedparrot.com/en/latest/results.html).

In [19]:
(results
 .sort_by("conversation_index", "index")
 .select("conversation_index", "index", "agent_name", "dialogue")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario            ┃ scenario ┃ scenario     ┃ answer                                                          ┃
┃ .conversation_index ┃ .index   ┃ .agent_name  ┃ .dialogue                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0                   │ 0        │ Interviewer  │ Hi there! Thanks for taking the time to speak with me today     │
│                     │          │              │ about the home construction newsletter. I see from your survey  │
│                     │          │              │ responses that you read the newsletter weekly. That's great to  │
│                     │          │              │ hear! I'd love to dive a bit deeper into your feedback. Can you │
│                     │          │              │ tell me more about what you enjoy most about the "Construction  │
│                     │          │              │ Tips" section and how it has been useful to you?                │
├─────────────────────┼──────────┼──────────────┼─────────────────────────────────────────────────────────────────┤
│ 0                   │ 1        │ Respondent 1 │ Hi! Absolutely, I'd be happy to elaborate. The "Construction    │
│                     │          │              │ Tips" section is incredibly valuable to me because it offers    │
│                     │          │              │ practical, actionable advice that I can apply directly to my    │
│                     │          │              │ projects. The tips are often very detailed and cover a wide     │
│                     │          │              │ range of topics, which helps me stay informed about best        │
│                     │          │              │ practices and new techniques in the industry. For example, I    │
│                     │          │              │ recently used some of the advice on optimizing insulation       │
│                     │          │              │ installation, which not only improved the energy efficiency of  │
│                     │          │              │ the building but also saved on costs. It’s these kinds of       │
│                     │          │              │ insights that make the section so useful.                       │
├─────────────────────┼──────────┼──────────────┼─────────────────────────────────────────────────────────────────┤
│ 0                   │ 2        │ Interviewer  │ That's fantastic to hear! It sounds like the "Construction      │
│                     │          │              │ Tips" section really provides you with some tangible benefits.  │
│                     │          │              │ You mentioned in your survey that you'd like to see more        │
│                     │          │              │ in-depth guides on sustainable building practices and case      │
│                     │          │              │ studies. Could you share a bit more about what specific aspects │
│                     │          │              │ of sustainable building you're most interested in, and what     │
│                     │          │              │ types of projects you'd like to see featured in the case        │
│                     │          │              │ studies?                                                        │
├─────────────────────┼──────────┼──────────────┼─────────────────────────────────────────────────────────────────┤
│ 0                   │ 3        │ Respondent 1 │ Certainly! When it comes to sustainable building practices, I'm │
│                     │          │              │ particularly interested in techniques that improve energy       │
│                     │          │              │ efficiency, reduce waste, and utilize eco-friendly materials.   │
│                     │          │              │ For ex

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we demonstrate how to post this notebook:

In [20]:
from edsl import Notebook

In [21]:
n = Notebook(path = "agentifying_responses.ipynb")

In [22]:
n.push(description = "Example code for creating agents for survey respondents and simulating follow-on interviews", visibility = "public")

{'description': 'Example code for creating agents for survey respondents and simulating follow-on interviews',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/2932971d-2e5a-42de-b5a7-ccbbfcea7247',
 'uuid': '2932971d-2e5a-42de-b5a7-ccbbfcea7247',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [23]:
n = Notebook(path = "agentifying_responses.ipynb")

In [24]:
n.patch(uuid = "2932971d-2e5a-42de-b5a7-ccbbfcea7247", value = n)

{'status': 'success'}